# Post-processing of EBSD indexing results from an Al-steel joint

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

Indexing = dictionary indexing + orientation refinement.

See the relevant package documentation for more details on the packages used here:
* diffsims: https://diffsims.readthedocs.io/en/latest/
* kikuchipy: https://kikuchipy.org/en/stable/
* hyperspy: hyperspy.org/hyperspy-doc/current/
* pyebsdindex: https://pyebsdindex.readthedocs.io/en/latest/
* orix: https://orix.readthedocs.io/en/stable/

Import required packages and print their versions

In [14]:
# Replace "inline" with "qt5" from the pyqt package for interactive plotting
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

import kikuchipy as kp
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from orix import io
from orix.quaternion import Orientation
import skimage.color as skc


# Directories
# Dataset naming (a-c) = (I-III)
dset = "c"
dir_mp = "/home/hakon/kode/emsoft/emdata/crystal_data"
dir_data = os.path.join("/home/hakon/phd/data/tina", dset)
dir_kp = os.path.join(dir_data, "kp")
dir_merged = os.path.join(dir_kp, "merged")

# Matplotlib
plt.rcParams.update({"figure.figsize": (5, 5), "font.size": 15})
savefig_kwds = dict(bbox_inches="tight", pad_inches=0, dpi=150)
plot_kwds = dict(
    scalebar_properties=dict(
        box_alpha=0, location="lower right", scale_loc="top"
    )
)

print("Run date: ", date.today())
print("\nSoftware versions\n------------------")
for pkg in [
    "kikuchipy",
    "matplotlib",
    "numpy",
    "orix",
    "scikit-image"
]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-05-31

Software versions
------------------
kikuchipy : 0.6.dev0
matplotlib : 3.5.2
numpy : 1.21.6
orix : 0.9.0
scikit-image : 0.19.2


Load single phase indexing results

In [15]:
phases = {
    "al": "Al",
    "alpha_alfesi2": r"$\alpha_c$",
    "ferrite": r"$\alpha$-Fe",
    "fe2al5": r"$\eta$",
    "fe4al13": r"$\theta$",
}

In [16]:
xmaps_ref = []
for phase_name, phase_name_alt in phases.items():
    fname = f"xmap_ref_{phase_name}.h5"
    xmap = io.load(os.path.join(dir_kp, phase_name, fname))
    xmap.phases[phase_name].name = phase_name_alt
    xmaps_ref.append(xmap)

Merge into multiphase crystal map by selecting the phase with the highest
normalized cross-correlation (NCC) score in each map point

In [17]:
xmap_merged = kp.indexing.merge_crystal_maps(
    crystal_maps=xmaps_ref, mean_n_best=1,
)
xmap_merged

Phase    Orientations         Name  Space group  Point group  Proper point group       Color
    0    4929 (23.8%)           Al        Fm-3m         m-3m                 432    tab:blue
    1    4064 (19.7%)   $\alpha_c$         Im-3          m-3                  23  tab:orange
    2   10168 (49.2%)  $\alpha$-Fe        Im-3m         m-3m                 432   tab:green
    3       91 (0.4%)       $\eta$         Cmcm          mmm                 222     tab:red
    4     1420 (6.9%)     $\theta$         C2/m          2/m                 112  tab:purple
Properties: scores, merged_scores
Scan unit: um

Read average dot product map to use for thresholding of incorrectly indexing
points in `MTEX`

In [18]:
maps_mean = plt.imread(os.path.join(dir_kp, "maps_mean.png"))
maps_mean = skc.rgb2gray(maps_mean[..., :3])
xmap_merged.prop["mean_intensity"] = maps_mean.ravel()

Write multiphase crystal map to file

In [19]:
io.save(os.path.join(dir_merged, "xmap_merged.h5"), xmap_merged, overwrite=True)
io.save(
    os.path.join(dir_merged, "xmap_merged.ang"),
    xmap_merged,
    confidence_index_prop="scores",
    extra_prop=["mean_intensity"],
    overwrite=True
)

Plot histogram of NCC scores

In [88]:
fig, ax = plt.subplots()
ax.hist(xmap_merged.scores, bins=50);
ax.set_xlabel("Normalized cross-correlation (NCC) scores")
ax.set_ylabel("Frequency")
fig.tight_layout()
fig.savefig(os.path.join(dir_merged, "hist_ncc.png"), **savefig_kwds)

Plot map of NCC scores

In [89]:
fig = xmap_merged.plot(
    "scores",
    cmap="gray",
    scalebar=False,
#    colorbar=True,
#    colorbar_label="NCC scores",
    return_figure=True,
    remove_padding=True,
    **plot_kwds
)

# Custom colorbar below map
ax = fig.axes[0]
divider = make_axes_locatable(ax)
cax = divider.append_axes(position="bottom", size="10%", pad=0.05)
cbar = fig.colorbar(ax.images[0], cax=cax, orientation="horizontal")

fig.tight_layout()
fig.savefig(os.path.join(dir_merged, "maps_ncc.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_merged, "maps_ncc.svg"), **savefig_kwds)

Plot map of NCC scores without a scalebar and colorbar

In [40]:
fig = xmap_merged.plot(
    "scores",
    cmap="gray",
    return_figure=True,
    remove_padding=True,
    scalebar=False
)
fig.tight_layout()
fig.savefig(os.path.join(dir_merged, "maps_ncc_no_scalebar.png"), **savefig_kwds)

Plot a phase map

In [41]:
fig = xmap_merged.plot(
    legend_properties=dict(loc="lower center", ncol=5, columnspacing=0.5),
    return_figure=True,
    remove_padding=True,
    scalebar_properties=dict(
        box_alpha=0, location="lower right", scale_loc="top"
    ),
)
fig.tight_layout()
fig.savefig(os.path.join(dir_merged, "maps_phase.png"), **savefig_kwds)

Close all plots

In [42]:
plt.close("all")